In [2]:
import tkinter as tk
from tkinter import StringVar, filedialog, messagebox
import cv2
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
import os

desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
model_load_path = os.path.join(desktop_path, "unet_model.keras")

model = tf.keras.models.load_model(model_load_path)
model.summary()

def upload_image():
    global img_cv2, img_tk, modified_img
    file_path = filedialog.askopenfilename()
    if file_path:
        img_cv2 = cv2.imread(file_path)
        img_cv2 = cv2.resize(img_cv2, (400, 300), interpolation=cv2.INTER_LANCZOS4)
        img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk = ImageTk.PhotoImage(img_pil)
        imglabel1.config(image=img_tk)
        imglabel1.image = img_tk
        modified_img = img_cv2  # Set modified_img to the original image initially

def save_modified_image():
    global modified_img
    if modified_img is not None:
        # Ask user for file path to save
        file_path = filedialog.asksaveasfilename(defaultextension=".png",
                                               filetypes=[("PNG files", "*.png"),
                                                          ("JPEG files", "*.jpg"),
                                                          ("All files", "*.*")])
        if file_path:
            cv2.imwrite(file_path, modified_img)
            messagebox.showinfo("Image Saved", f"Image saved as {file_path}")
    else:
        messagebox.showwarning("No Image", "No modified image to save.")

def resize():
    global img_cv2, modified_img
    size_str = size_var.get()
    size = float(size_str)
    if (size > 100 or size < 0):
       messagebox.showwarning("Invalid Input", "Please enter a valid percentage.")
    else:
        
        # Calculate new dimensions
        new_width = int(img_cv2.shape[1] * size/100)
        new_height = int(img_cv2.shape[0] * size/100)

        # Resize the image
        modified_img = cv2.resize(img_cv2, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)

        # Convert to ImageTk.PhotoImage and update imglabel2
        img_rgb = cv2.cvtColor(modified_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_resized = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_resized)
        imglabel2.image = img_tk_resized

def rotate():
    global img_cv2, modified_img
    angle_str = angle_var.get()
    try:
        angle = float(angle_str)
    except ValueError:
        messagebox.showwarning("Invalid Input", "Please enter a valid numeric angle.")
        return
    
    if angle < 0 or angle > 360:
        messagebox.showwarning("Invalid Input", "Please enter a valid numeric angle between 0 and 360.")
        return

    if img_cv2 is not None:
        # Get image dimensions
        (h, w) = img_cv2.shape[:2]
        center = (w / 2, h / 2)

        # Calculate rotation matrix and rotate the image
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated_img = cv2.warpAffine(img_cv2, M, (w, h))

        # Crop the image to remove black borders (Optional)
        # Get the size of the rotated image to determine the bounding box
        cos_angle = np.abs(M[0, 0])
        sin_angle = np.abs(M[0, 1])

        # Compute the new bounding dimensions
        new_w = int((h * sin_angle) + (w * cos_angle))
        new_h = int((h * cos_angle) + (w * sin_angle))

        # Adjust the rotation matrix to account for the change in dimension
        M[0, 2] += (new_w / 2) - center[0]
        M[1, 2] += (new_h / 2) - center[1]

        # Perform the rotation with the adjusted matrix
        rotated_img = cv2.warpAffine(img_cv2, M, (new_w, new_h))

        # Resize the image to original size if needed
        rotated_img = cv2.resize(rotated_img, (w, h))

        # Update modified_img
        modified_img = rotated_img

        # Convert to ImageTk.PhotoImage and update imglabel2
        img_rgb = cv2.cvtColor(modified_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_rotated = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_rotated)
        imglabel2.image = img_tk_rotated
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def flip_vertical():
    global img_cv2, modified_img
    if img_cv2 is not None:
        verticalflip_img = cv2.flip(img_cv2, 0)  

        modified_img = verticalflip_img

        img_rgb = cv2.cvtColor(modified_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_verticalflipped = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_verticalflipped)
        imglabel2.image = img_tk_verticalflipped
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def flip_horizontal():
    global img_cv2, modified_img
    if img_cv2 is not None:
        horizontalflip_img = cv2.flip(img_cv2, 1)  

        modified_img = horizontalflip_img

        img_rgb = cv2.cvtColor(modified_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_horizontalflipped = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_horizontalflipped)
        imglabel2.image = img_tk_horizontalflipped
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def invert_colors():
    global img_cv2, modified_img
    if img_cv2 is not None:
        inverted_image = cv2.bitwise_not(img_cv2)
        
        modified_img = inverted_image

        img_rgb = cv2.cvtColor(inverted_image, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_inverted = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_inverted)
        imglabel2.image = img_tk_inverted
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def convert_to_greyscale():
    global img_cv2, modified_img
    if img_cv2 is not None:
        # Convert the image to greyscale
        greyscale_img = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)
        # Convert to 3-channel image for display purposes
        greyscale_img = cv2.cvtColor(greyscale_img, cv2.COLOR_GRAY2RGB)
        
        # Update modified_img
        modified_img = greyscale_img
        
        # Convert to ImageTk.PhotoImage and update imglabel2
        img_pil = Image.fromarray(greyscale_img)
        img_tk_greyscale = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_greyscale)
        imglabel2.image = img_tk_greyscale
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def convert_to_bw():
    global img_cv2, modified_img
    if img_cv2 is not None:
        # Convert the image to greyscale
        greyscale_image = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)

        # apply black and white threshold
        _, bw_img = cv2.threshold(greyscale_image, 127, 255, cv2.THRESH_BINARY)
        
        # Update modified_img
        modified_img = bw_img
        
        # Convert to ImageTk.PhotoImage and update imglabel2
        img_pil = Image.fromarray(bw_img)
        img_tk_bw = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_bw)
        imglabel2.image = img_tk_bw
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def convert_to_color():
    global img_cv2, modified_img
    if img_cv2 is not None:
        # Convert the image to RGB
        RGB_img = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
        
        # Update modified_img
        modified_img = RGB_img
        
        # Convert to ImageTk.PhotoImage and update imglabel2
        img_pil = Image.fromarray(RGB_img)
        img_tk_RGB = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_RGB)
        imglabel2.image = img_tk_RGB
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def watershed_segment():
    global img_cv2, modified_img
    if img_cv2 is not None:
        #convert to grayscale
        gray = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)

        #add binary threshold
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        #remove noise using morphological operations
        kernel = np.ones((3, 3), np.uint8)
        opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)

        #determine the sure background area
        sure_bg = cv2.dilate(opening, kernel, iterations=3)

        #Find the sure foreground area
        dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
        _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

        #find the unknown region
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(sure_bg, sure_fg)

        #labelling markers
        _, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        markers[unknown == 255] = 0
        segmented_img = img_cv2.copy()
        markers = cv2.watershed(segmented_img, markers)
        segmented_img[markers == -1] = [0, 0, 255]

        #update modified image
        modified_img = segmented_img

        # Convert to ImageTk.PhotoImage and update imglabel2
        img_rgb = cv2.cvtColor(modified_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_segmented = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_segmented)
        imglabel2.image = img_tk_segmented
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def color_segment():
    global img_cv2, modified_img
    if img_cv2 is not None:
        # Convert the image from BGR to HSV color space
        hsv_img = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2HSV)

        # Define lower and upper bounds for the color to segment (e.g., red color)
        lower_bound = np.array([0, 50, 50])  # Lower bound for red color
        upper_bound = np.array([10, 255, 255])  # Upper bound for red color

        # Create a mask using the bounds
        mask = cv2.inRange(hsv_img, lower_bound, upper_bound)

        # Segment the image by combining the mask with the original image
        segmented_img = cv2.bitwise_and(img_cv2, img_cv2, mask=mask)

        # Update the modified image
        modified_img = segmented_img

        # Convert to ImageTk.PhotoImage and update imglabel2
        img_rgb = cv2.cvtColor(modified_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_segmented = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_segmented)
        imglabel2.image = img_tk_segmented
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def apply_filter(filter_type):
    global img_cv2, modified_img
    if img_cv2 is not None:
        if filter_type == 'min':
            filtered_img = cv2.erode(img_cv2, np.ones((3, 3), np.uint8))
        elif filter_type == 'max':
            filtered_img = cv2.dilate(img_cv2, np.ones((3, 3), np.uint8))
        elif filter_type == 'median':
            filtered_img = cv2.medianBlur(img_cv2, 3)
        elif filter_type == 'gaussian':
            filtered_img = cv2.GaussianBlur(img_cv2, (3, 3), 1.0)
        else:
            return
        
        modified_img = filtered_img

        img_rgb = cv2.cvtColor(filtered_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_filtered = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_filtered)
        imglabel2.image = img_tk_filtered
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def gamma_correction():
    global img_cv2, modified_img
    if img_cv2 is not None:
        gamma = 1.5  # Example gamma value, can be adjustable
        look_up_table = np.array([((i / 255.0) ** gamma) * 255 for i in range(256)]).astype("uint8")
        corrected_img = cv2.LUT(img_cv2, look_up_table)

        modified_img = corrected_img

        img_rgb = cv2.cvtColor(corrected_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_corrected = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_corrected)
        imglabel2.image = img_tk_corrected
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def histogram_equalization():
    global img_cv2, modified_img
    if img_cv2 is not None:
        img_yuv = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2YUV)
        img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
        equalized_img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

        modified_img = equalized_img

        img_rgb = cv2.cvtColor(equalized_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_equalized = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_equalized)
        imglabel2.image = img_tk_equalized
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def white_balance():
    global img_cv2, modified_img
    if img_cv2 is not None:
        # Convert image to float32 for more precise computation
        img_float = img_cv2.astype(np.float32)

        # Calculate the average color of the image
        avg_color = np.mean(img_float, axis=(0, 1))

        # Scale each channel to make the average color neutral (white)
        white_balance_scale = np.mean(avg_color) / avg_color
        white_balanced_img = img_float * white_balance_scale

        # Clip the values to be in valid range [0, 255] and convert back to uint8
        white_balanced_img = np.clip(white_balanced_img, 0, 255).astype(np.uint8)

        modified_img = white_balanced_img

        img_rgb = cv2.cvtColor(white_balanced_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_white_balanced = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_white_balanced)
        imglabel2.image = img_tk_white_balanced
    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def open_crop_options():
    global img_cv2, modified_img

    if img_cv2 is not None:
        # Create a popup window
        crop_popup = tk.Toplevel()
        crop_popup.title("Crop Options")
        crop_popup.geometry("300x250")
        crop_popup.resizable(False, False)

        # Variables for user input
        crop_size_var = tk.IntVar(value=50)  # Default crop size (in percentage)
        crop_start_var = tk.StringVar(value="center")  # Default starting point

        # Crop Size Label and Entry
        tk.Label(crop_popup, text="Crop Size (%):", font=font_medium).pack(pady=10)
        crop_size_entry = tk.Entry(crop_popup, textvariable=crop_size_var, font=font_medium, width=10)
        crop_size_entry.pack(pady=5)

        # Starting Point Label and Radio Buttons
        tk.Label(crop_popup, text="Starting Point:", font=font_medium).pack(pady=10)
        
        points_frame = tk.Frame(crop_popup)
        points_frame.pack(pady=5)
        tk.Radiobutton(points_frame, text="Center", variable=crop_start_var, value="center", font=font_medium).grid(row=0, column=0)
        tk.Radiobutton(points_frame, text="Top", variable=crop_start_var, value="top", font=font_medium).grid(row=0, column=1)
        tk.Radiobutton(points_frame, text="Bottom", variable=crop_start_var, value="bottom", font=font_medium).grid(row=0, column=2)
        tk.Radiobutton(points_frame, text="Left", variable=crop_start_var, value="left", font=font_medium).grid(row=1, column=0)
        tk.Radiobutton(points_frame, text="Right", variable=crop_start_var, value="right", font=font_medium).grid(row=1, column=1)

        # Crop Image Button
        tk.Button(crop_popup, text="Crop Image", font=font_medium, bg=button_bg, fg=button_fg, 
                  cursor="hand2", command=lambda: crop_image(crop_size_var.get(), crop_start_var.get(), crop_popup)).pack(pady=20)

    else:
        messagebox.showwarning("No Image", "Please upload an image first.")


def crop_image(crop_size_percent, start_point, popup):
    global img_cv2, modified_img

    if img_cv2 is not None:
        height, width = img_cv2.shape[:2]

        # Calculate crop size based on user input percentage
        crop_width = int(width * (crop_size_percent / 100))
        crop_height = int(height * (crop_size_percent / 100))

        # Determine the starting point based on the user’s selection
        if start_point == "center":
            x_start = (width - crop_width) // 2
            y_start = (height - crop_height) // 2
        elif start_point == "top":
            x_start = (width - crop_width) // 2
            y_start = 0
        elif start_point == "bottom":
            x_start = (width - crop_width) // 2
            y_start = height - crop_height
        elif start_point == "left":
            x_start = 0
            y_start = (height - crop_height) // 2
        elif start_point == "right":
            x_start = width - crop_width
            y_start = (height - crop_height) // 2
        else:
            x_start = (width - crop_width) // 2
            y_start = (height - crop_height) // 2

        # Ensure the end points don't exceed image dimensions
        x_end = min(x_start + crop_width, width)
        y_end = min(y_start + crop_height, height)

        # Perform the cropping
        cropped_img = img_cv2[y_start:y_end, x_start:x_end]

        # Update global modified image
        modified_img = cropped_img

        # Convert cropped image to Tkinter-compatible format and update imglabel2
        img_rgb = cv2.cvtColor(modified_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk_cropped = ImageTk.PhotoImage(img_pil)
        imglabel2.config(image=img_tk_cropped)
        imglabel2.image = img_tk_cropped

        # Close the popup after cropping
        popup.destroy()

    else:
        messagebox.showwarning("No Image", "Please upload an image first.")

def semantic_segmentation():
    global img_cv2, modified_img
    if img_cv2 is not None:
        # Preprocess the image
        IMG_HEIGHT = 128
        IMG_WIDTH = 128

        # Convert BGR (OpenCV format) to RGB
        img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)

        # Resize the image to match the model's input size
        img_resized = cv2.resize(img_rgb, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_AREA)
        
        # Normalize the image to [0, 1]
        img_normalized = img_resized.astype(np.float32) / 255.0

        # Expand dimensions to match model's input shape (1, H, W, C)
        img_input = np.expand_dims(img_normalized, axis=0)  

        # Predict the mask using the trained model
        pred_mask = model.predict(img_input)[0]  
        
        # Convert predictions to class labels by taking the argmax across the channel dimension
        pred_mask_labels = np.argmax(pred_mask, axis=-1)  

        # Adjust colors as needed for better visualization
        class_colors = {
            0: [0, 0, 0],        
            1: [0, 255, 0],      
            2: [0, 0, 255],      
        }

        # Create an empty color mask
        color_mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)

        # Assign colors to each class in the mask
        for class_id, color in class_colors.items():
            color_mask[pred_mask_labels == class_id] = color

        # Resize the color mask back to the original image size
        color_mask_resized = cv2.resize(color_mask, (img_cv2.shape[1], img_cv2.shape[0]), interpolation=cv2.INTER_NEAREST)

        # Combine the original image with the color mask
        alpha = 0.7  
        beta = 0.3   
        gamma = 0    

        # Ensure both images are in the same color space (BGR for OpenCV)
        segmented_img = cv2.addWeighted(img_cv2, alpha, color_mask_resized, beta, gamma)

        # Update the global modified_img
        modified_img = segmented_img

        # Convert the segmented image to RGB for displaying in Tkinter
        img_display = cv2.cvtColor(segmented_img, cv2.COLOR_BGR2RGB)

        # Convert the NumPy array to a PIL Image
        img_pil = Image.fromarray(img_display)

        # Convert the PIL Image to a PhotoImage for Tkinter
        img_tk_segmented = ImageTk.PhotoImage(img_pil)

        # Update the image label in the GUI
        imglabel2.config(image=img_tk_segmented)
        imglabel2.image = img_tk_segmented

root = tk.Tk()
root.geometry("1366x768")
root.title("Photo Lab")

# Set background color
root.configure(bg="#2b2b2b")

# Define colors and fonts
button_bg = "#4b4b4b"
button_fg = "#ffffff"
section_bg = "#1e1e1e"
font_large = ("Arial", 18, "bold")
font_medium = ("Arial", 12, "bold")
font_small = ("Arial", 10, "normal")

# Header label
header = tk.Label(root, text="Welcome to Photo Lab", font=("Castellar", 28, "bold"), fg="#ffffff", bg="#2b2b2b")
header.pack(pady=20)

# Create frames for layout
left_frame = tk.Frame(root, bg=section_bg, bd=2, relief=tk.GROOVE)
left_frame.place(x=30, y=10, width=250, height=780)

center_frame = tk.Frame(root, bg=section_bg, bd=2, relief=tk.GROOVE)
center_frame.place(x=315, y=125, width=900, height=600)

right_frame = tk.Frame(root, bg=section_bg, bd=2, relief=tk.GROOVE)
right_frame.place(x=1250, y=10, width=250, height=780)

# Image label frames
image_frame1 = tk.LabelFrame(center_frame, text="Original Image", font=font_medium, fg=button_fg, bg=section_bg)
image_frame1.place(x=10, y=10, width=420, height=580)

image_frame2 = tk.LabelFrame(center_frame, text="Modified Image", font=font_medium, fg=button_fg, bg=section_bg)
image_frame2.place(x=465, y=10, width=420, height=580)

imglabel1 = tk.Label(image_frame1, bg=section_bg)
imglabel1.pack(expand=True)

imglabel2 = tk.Label(image_frame2, bg=section_bg)
imglabel2.pack(expand=True)

# Button placement on the left frame
tk.Label(left_frame, text="Basic Operations", font=font_large, fg=button_fg, bg=section_bg).pack(pady=10)

# Button placement on the right frame
tk.Label(right_frame, text="Advanced Features", font=font_large, fg=button_fg, bg=section_bg).pack(pady=10)

# Upload and Save Section
upload_save_frame = tk.LabelFrame(left_frame, text="Upload & Save", font=font_medium, fg=button_fg, bg=section_bg, bd=2, relief=tk.RIDGE)
upload_save_frame.pack(pady=10, fill=tk.X)

button2 = tk.Button(upload_save_frame, text="Upload Image", command=upload_image, font=font_medium, bg='green', fg=button_fg, cursor="hand2")
button2.pack(pady=5, fill=tk.X)

button3 = tk.Button(upload_save_frame, text="Save Image", command=save_modified_image, font=font_medium, bg='blue', fg=button_fg, cursor="hand2")
button3.pack(pady=5, fill=tk.X)

# Resize and Rotate Section
resize_rotate_frame = tk.LabelFrame(left_frame, text="Resize & Rotate", font=font_medium, fg=button_fg, bg=section_bg, bd=2, relief=tk.RIDGE)
resize_rotate_frame.pack(pady=10, fill=tk.X)

size_var = tk.StringVar()

size_label = tk.Label(resize_rotate_frame, text="Size (%)", font=font_small, fg=button_fg, bg=section_bg)
size_label.pack(pady=5)
size_entry = tk.Entry(resize_rotate_frame, textvariable=size_var, font=font_small)
size_entry.pack(pady=5, fill=tk.X)

button1 = tk.Button(resize_rotate_frame, text="Resize Image", command=resize, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button1.pack(pady=5, fill=tk.X)

angle_var = tk.StringVar()

angle_label = tk.Label(resize_rotate_frame, text="Angle (In Degrees)", font=font_small, fg=button_fg, bg=section_bg)
angle_label.pack(pady=5)
angle_entry = tk.Entry(resize_rotate_frame, textvariable=angle_var, font=font_small)
angle_entry.pack(pady=5, fill=tk.X)

sub_btn = tk.Button(resize_rotate_frame, text="Rotate Image", command=rotate, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
sub_btn.pack(pady=5, fill=tk.X)

button17 = tk.Button(resize_rotate_frame, text="Crop Options", command=open_crop_options, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button17.pack(pady=5, fill=tk.X)

button19 = tk.Button(resize_rotate_frame, text="Flip Vertical", command=flip_vertical, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button19.pack(pady=5, fill=tk.X)

button20 = tk.Button(resize_rotate_frame, text="Flip Horizontal", command=flip_horizontal, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button20.pack(pady=5, fill=tk.X)

# Color Conversion Section
color_conversion_frame = tk.LabelFrame(left_frame, text="Color Conversion", font=font_medium, fg=button_fg, bg=section_bg, bd=2, relief=tk.RIDGE)
color_conversion_frame.pack(pady=10, fill=tk.X)

button4 = tk.Button(color_conversion_frame, text="Convert to Binary", command=convert_to_bw, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button4.pack(pady=5, fill=tk.X)

button5 = tk.Button(color_conversion_frame, text="Convert to Color", command=convert_to_color, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button5.pack(pady=5, fill=tk.X)

button6 = tk.Button(color_conversion_frame, text="Convert to Greyscale", command=convert_to_greyscale, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button6.pack(pady=5, fill=tk.X)

button7 = tk.Button(color_conversion_frame, text="Invert Colors", command=invert_colors, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button7.pack(pady=5, fill=tk.X)

# Image Segmentation Section
segmentation_frame = tk.LabelFrame(right_frame, text="Image Segmentation", font=font_medium, fg=button_fg, bg=section_bg, bd=2, relief=tk.RIDGE)
segmentation_frame.pack(pady=10, fill=tk.X)

button8 = tk.Button(segmentation_frame, text="Watershed Segmentation", command=watershed_segment, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button8.pack(pady=5, fill=tk.X)

button16 = tk.Button(segmentation_frame, text="Color Based Segmentation", command=color_segment, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button16.pack(pady=5, fill=tk.X)

# Filtering Section
filtering_frame = tk.LabelFrame(right_frame, text="Filtering", font=font_medium, fg=button_fg, bg=section_bg, bd=2, relief=tk.RIDGE)
filtering_frame.pack(pady=10, fill=tk.X)

button9 = tk.Button(filtering_frame, text="Minimum Filter", command=lambda: apply_filter('min'), font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button9.pack(pady=5, fill=tk.X)

button10 = tk.Button(filtering_frame, text="Maximum Filter", command=lambda: apply_filter('max'), font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button10.pack(pady=5, fill=tk.X)

button11 = tk.Button(filtering_frame, text="Median Filter", command=lambda: apply_filter('median'), font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button11.pack(pady=5, fill=tk.X)

button12 = tk.Button(filtering_frame, text="Gaussian Blur Filter", command=lambda: apply_filter('gaussian'), font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button12.pack(pady=5, fill=tk.X)

# Color Transformation Section
color_transformation_frame = tk.LabelFrame(right_frame, text="Color Transformation", font=font_medium, fg=button_fg, bg=section_bg, bd=2, relief=tk.RIDGE)
color_transformation_frame.pack(pady=10, fill=tk.X)

button13 = tk.Button(color_transformation_frame, text="Gamma Correction", command=gamma_correction, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button13.pack(pady=5, fill=tk.X)

button14 = tk.Button(color_transformation_frame, text="Histogram Equalization", command=histogram_equalization, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button14.pack(pady=5, fill=tk.X)

button15 = tk.Button(color_transformation_frame, text="White Balance", command=white_balance, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button15.pack(pady=5, fill=tk.X)

#Deep Learning features
deep_learning_frame = tk.LabelFrame(right_frame, text="Deep Learning Features", font=font_medium, fg=button_fg, bg=section_bg, bd=2, relief=tk.RIDGE)
deep_learning_frame.pack(pady=10, fill=tk.X)

button18 = tk.Button(deep_learning_frame, text="Semantic Segmentation", command=semantic_segmentation, font=font_medium, bg=button_bg, fg=button_fg, cursor="hand2")
button18.pack(pady=5, fill=tk.X)

root.mainloop()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 128, 128,  │      1,792 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 128, 128,  │     36,928 │ conv2d_19[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 64, 64,    │          0 │ conv2d_20[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 64, 64,    │     73,856 │ max_pooling2d_4[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 64, 64,    │    147,584 │ conv2d_21[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 32, 32,    │          0 │ conv2d_22[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 32, 32,    │    295,168 │ max_pooling2d_5[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 32, 32,    │    590,080 │ conv2d_23[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 16, 16,    │          0 │ conv2d_24[0][0]   │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_25 (Conv2D)  │ (None, 16, 16,    │  1,180,160 │ max_pooling2d_6[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_26 (Conv2D)  │ (None, 16, 16,    │  2,359,808 │ conv2d_25[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_7     │ (None, 8, 8, 512) │          0 │ conv2d_26[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_27 (Conv2D)  │ (None, 8, 8,      │  4,719,616 │ max_pooling2d_7[… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_28 (Conv2D)  │ (None, 8, 8,      │  9,438,208 │ conv2d_27[0][0]   │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_4  │ (None, 16, 16,    │  2,097,664 │ conv2d_28[0][0]   │
│ (Conv2DTranspose)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 16, 16,    │          0 │ conv2d_26[0][0],

 Total params: 93,095,627 (355.13 MB)

 Trainable params: 31,031,875 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 62,063,752 (236.75 MB)